In [1]:
# Essentials
import os, sys, glob
import pandas as pd
import numpy as np
import nibabel as nib
import scipy.io as sio
from tqdm import tqdm

# Stats
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import pingouin as pg

# Plotting
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['svg.fonttype'] = 'none'

In [2]:
from networkx import from_numpy_matrix, degree_centrality, closeness_centrality, betweenness_centrality, subgraph_centrality

/Users/lindenmp/opt/anaconda3/envs/neurodev_cs_predictive/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.3.7, the latest is 0.3.8.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


In [3]:
sys.path.append('/Users/lindenmp/Google-Drive-Penn/work/research_projects/neurodev_cs_predictive/1_code/')
from func import set_proj_env, rank_int, node_strength, ave_control

In [4]:
parc_str = 'schaefer' # 'schaefer' 'lausanne' 'glasser'
parc_scale = 200 # 200/400 | 125/250 | 360
edge_weight = 'streamlineCount' # 'streamlineCount' 'volNormStreamline'
parcel_names, parcel_loc, drop_parcels, num_parcels = set_proj_env(parc_str = parc_str, parc_scale = parc_scale, edge_weight = edge_weight)

In [5]:
if parc_str == 'schaefer' or parc_str == 'glasser':
    exclude_str = 't1Exclude'
else:
    exclude_str = 'fsFinalExclude'

In [6]:
# output file prefix
outfile_prefix = parc_str+'_'+str(parc_scale)+'_'+edge_weight+'_'
outfile_prefix

'schaefer_200_streamlineCount_'

In [7]:
# we want to calculate conn features including subcortex
# drop brainstem but retain subcortex.
if parc_str == 'lausanne':
    num_parcels = len(parcel_loc[parcel_loc != 2])

### Setup directory variables

In [8]:
print(os.environ['PIPELINEDIR'])
if not os.path.exists(os.environ['PIPELINEDIR']): os.makedirs(os.environ['PIPELINEDIR'])

/Users/lindenmp/Google-Drive-Penn/work/research_projects/neurodev_cs_predictive/2_pipeline


In [9]:
storedir = os.path.join(os.environ['PIPELINEDIR'], '1_compute_node_features', 'store')
print(storedir)
if not os.path.exists(storedir): os.makedirs(storedir)

outputdir = os.path.join(os.environ['PIPELINEDIR'], '1_compute_node_features', 'out')
print(outputdir)
if not os.path.exists(outputdir): os.makedirs(outputdir)

/Users/lindenmp/Google-Drive-Penn/work/research_projects/neurodev_cs_predictive/2_pipeline/1_compute_node_features/store
/Users/lindenmp/Google-Drive-Penn/work/research_projects/neurodev_cs_predictive/2_pipeline/1_compute_node_features/out


In [10]:
figdir = os.path.join(os.environ['OUTPUTDIR'], 'figs')
print(figdir)
if not os.path.exists(figdir): os.makedirs(figdir)

/Users/lindenmp/Google-Drive-Penn/work/research_projects/neurodev_cs_predictive/3_output/figs


## Load data

In [11]:
# Load data
df = pd.read_csv(os.path.join(os.environ['PIPELINEDIR'], '0_get_sample', 'out', exclude_str+'_df.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)
print(df.shape)

(1100, 51)


In [12]:
# Missing data file for this subject only for schaefer 200
if parc_scale == 200:
    df.drop(labels = (112598, 5161), inplace=True)

In [13]:
# output dataframe
str_labels = ['str_' + str(i) for i in range(num_parcels)]
ac_labels = ['ac_' + str(i) for i in range(num_parcels)]
bc_labels = ['bc_' + str(i) for i in range(num_parcels)]
cc_labels = ['cc_' + str(i) for i in range(num_parcels)]
sgc_labels = ['sgc_' + str(i) for i in range(num_parcels)]

df_node = pd.DataFrame(index = df.index, columns = str_labels + ac_labels + bc_labels + cc_labels + sgc_labels)
print(df_node.shape)

(1099, 1000)


## Load in structural connectivity matrices

In [14]:
# subject filter
subj_filt = np.zeros((df.shape[0],)).astype(bool)

In [15]:
print(os.environ['CONN_STR'])

A = np.zeros((num_parcels, num_parcels, df.shape[0]))
for (i, (index, row)) in enumerate(df.iterrows()):
    file_name = os.environ['SC_NAME_TMP'].replace("scanid", str(index[1]))
    file_name = file_name.replace("bblid", str(index[0]))
    full_path = glob.glob(os.path.join(os.environ['SCDIR'], file_name))
    if i == 0: print(full_path)
    if len(full_path) > 0:
        mat_contents = sio.loadmat(full_path[0])
        a = mat_contents[os.environ['CONN_STR']]
        if parc_str == 'lausanne': # drop brainstem but retain subcortex.
            a = a[parcel_loc != 2,:]
            a = a[:,parcel_loc != 2]
        A[:,:,i] = a
    elif len(full_path) == 0:
        print(file_name + ': NOT FOUND')
        subj_filt[i] = True
        A[:,:,i] = np.full((num_parcels, num_parcels), np.nan)

connectivity
['/Volumes/work_ssd/research_data/PNC/processedData/diffusion/deterministic_20171118/81287/20100114x2738/tractography/connectivity/81287_20100114x2738_SchaeferPNC_200_dti_streamlineCount_connectivity.mat']
82051/*x2856/tractography/connectivity/82051_*x2856_SchaeferPNC_200_dti_streamlineCount_connectivity.mat: NOT FOUND
87804/*x3144/tractography/connectivity/87804_*x3144_SchaeferPNC_200_dti_streamlineCount_connectivity.mat: NOT FOUND
91332/*x3362/tractography/connectivity/91332_*x3362_SchaeferPNC_200_dti_streamlineCount_connectivity.mat: NOT FOUND
87990/*x3676/tractography/connectivity/87990_*x3676_SchaeferPNC_200_dti_streamlineCount_connectivity.mat: NOT FOUND
103737/*x3964/tractography/connectivity/103737_*x3964_SchaeferPNC_200_dti_streamlineCount_connectivity.mat: NOT FOUND
87470/*x4000/tractography/connectivity/87470_*x4000_SchaeferPNC_200_dti_streamlineCount_connectivity.mat: NOT FOUND
104161/*x4104/tractography/connectivity/104161_*x4104_SchaeferPNC_200_dti_streamlin

In [16]:
np.sum(subj_filt)

19

In [17]:
if any(subj_filt):
    A = A[:,:,~subj_filt]
    df = df.loc[~subj_filt]
    df_node = df_node.loc[~subj_filt]
print(df_node.shape)

(1080, 1000)


### Check if any subjects have disconnected nodes in A matrix

In [18]:
# subject filter
subj_filt = np.zeros((df.shape[0],)).astype(bool)

In [19]:
for i in range(A.shape[2]):
    if np.any(np.sum(A[:,:,i], axis = 1) == 0):
        subj_filt[i] = True

In [20]:
np.sum(subj_filt)

12

In [21]:
if any(subj_filt):
    A = A[:,:,~subj_filt]
    df = df.loc[~subj_filt]
    df_node = df_node.loc[~subj_filt]
print(df_node.shape)

(1068, 1000)


In [22]:
np.sum(df['averageManualRating'] == 2)

924

In [23]:
np.sum(df['dti64QAManualScore'] == 2)

655

### Get streamline count and network density

In [24]:
A_c = np.zeros((A.shape[2],))
A_d = np.zeros((A.shape[2],))
for i in range(A.shape[2]):
    A_c[i] = np.sum(np.triu(A[:,:,i]))
    A_d[i] = np.count_nonzero(np.triu(A[:,:,i]))/((A[:,:,i].shape[0]**2-A[:,:,i].shape[0])/2)
df.loc[:,'streamline_count'] = A_c
df.loc[:,'network_density'] = A_d

### Compute node metrics

In [25]:
# fc stored as 3d matrix, subjects of 3rd dim
S = np.zeros((df.shape[0], num_parcels))
AC = np.zeros((df.shape[0], num_parcels))
BC = np.zeros((df.shape[0], num_parcels))
CC = np.zeros((df.shape[0], num_parcels))
SGC = np.zeros((df.shape[0], num_parcels))

# for (i, (index, row)) in enumerate(df.iterrows()):
for i in tqdm(np.arange(df.shape[0])):
    S[i,:] = node_strength(A[:,:,i])
    AC[i,:] = ave_control(A[:,:,i])
    G = from_numpy_matrix(A[:,:,i])
    BC[i,:] = np.array(list(betweenness_centrality(G, normalized=False).values()))
    CC[i,:] = np.array(list(closeness_centrality(G).values()))
    SGC[i,:] = np.array(list(subgraph_centrality(G).values()))
    
df_node.loc[:,str_labels] = S
df_node.loc[:,ac_labels] = AC
df_node.loc[:,bc_labels] = BC
df_node.loc[:,cc_labels] = CC
df_node.loc[:,sgc_labels] = SGC

100%|██████████| 1068/1068 [05:36<00:00,  3.17it/s]


## Recalculate average control at different C params

In [26]:
c_params = np.array([10, 100, 1000, 10000])
c_params

array([   10,   100,  1000, 10000])

In [27]:
# output dataframe
df_node_ac_overc = pd.DataFrame(index = df.index)

for c in c_params:
    print(c)
    ac_labels_new = ['ac_c' + str(c) + '_' + str(i) for i in range(num_parcels)]
    df_node_ac_temp = pd.DataFrame(index = df.index, columns = ac_labels_new)
    
    # fc stored as 3d matrix, subjects of 3rd dim
    AC = np.zeros((df.shape[0], num_parcels))
    for i in tqdm(np.arange(df.shape[0])):
        AC[i,:] = ave_control(A[:,:,i], c = c)

    df_node_ac_temp.loc[:,ac_labels_new] = AC
    df_node_ac_overc = pd.concat((df_node_ac_overc, df_node_ac_temp), axis = 1)

  0%|          | 4/1068 [00:00<00:26, 39.58it/s]

10


  0%|          | 5/1068 [00:00<00:25, 41.03it/s]

100


  0%|          | 5/1068 [00:00<00:25, 40.94it/s]

1000


  0%|          | 5/1068 [00:00<00:25, 40.95it/s]

10000


100%|██████████| 1068/1068 [00:27<00:00, 39.01it/s]


## Scale average controllability to test for differences in initial conditions

In [28]:
df_node_ac_i2 = df_node.loc[:,ac_labels] * (2**2)
df_node_ac_i2.head()

,,ac_0,ac_1,ac_2,ac_3,ac_4,ac_5,ac_6,ac_7,ac_8,ac_9,...,ac_190,ac_191,ac_192,ac_193,ac_194,ac_195,ac_196,ac_197,ac_198,ac_199
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,4.43468,4.10575,4.13392,4.04356,4.04503,4.11175,4.06094,4.17129,4.01457,4.05663,...,4.20857,23.4328,18.1331,4.16028,4.09473,4.17991,4.25065,4.05589,4.07651,4.08423
80680,2739,4.46303,4.03204,9.40825,4.20029,4.02867,4.16083,4.14702,4.43658,4.04873,4.45214,...,38.215,490.179,1245.51,77.22,8.59439,11.0215,16.0148,5.31901,7.7746,9.27361
81754,2740,4.24757,4.05902,4.05762,4.05663,4.06498,4.13941,4.07664,4.02832,4.0307,4.16414,...,4.4644,4.24664,7.12033,4.4122,4.22798,4.13203,4.06124,4.0094,4.07202,4.01678
81903,2749,4.13147,4.00683,4.04414,4.05523,4.03695,4.0369,4.01868,4.02167,4.02456,4.06423,...,4.25671,4.38306,62.7773,4.23304,4.08596,4.18215,4.13089,4.01645,4.06997,4.025
81043,2750,4.17039,4.11102,4.78324,4.10579,4.13066,4.16035,4.05459,4.01947,4.01141,4.05958,...,4.92372,4.49082,7.49184,4.22606,4.08274,4.16158,4.1003,4.0523,4.12231,4.05621


In [29]:
df_node_ac_overc_i2 = df_node_ac_overc * (2**2)
df_node_ac_overc_i2.head()

,,ac_c10_0,ac_c10_1,ac_c10_2,ac_c10_3,ac_c10_4,ac_c10_5,ac_c10_6,ac_c10_7,ac_c10_8,ac_c10_9,...,ac_c10000_190,ac_c10000_191,ac_c10000_192,ac_c10000_193,ac_c10000_194,ac_c10000_195,ac_c10000_196,ac_c10000_197,ac_c10000_198,ac_c10000_199
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,4.160120,4.092036,4.093107,4.040229,4.037758,4.084432,4.053120,4.109415,4.013584,4.044086,...,4.027053,4.106296,4.070072,4.021275,4.003737,4.020574,4.030353,4.007394,4.011361,4.007299
80680,2739,4.252323,4.015835,4.607413,4.054295,4.027314,4.116766,4.110584,4.188318,4.020290,4.147870,...,4.027945,4.057771,4.202422,4.095051,4.024277,4.035263,4.028265,4.009764,4.029263,4.011326
81754,2740,4.234071,4.055776,4.055097,4.054822,4.059832,4.079835,4.075693,4.027565,4.029863,4.157487,...,4.053892,4.009598,4.040507,4.049698,4.022206,4.017173,4.008040,4.001297,4.008790,4.002280
81903,2749,4.127245,4.006670,4.041787,4.054342,4.036801,4.032573,4.018572,4.021521,4.024475,4.063946,...,4.030197,4.009415,4.080617,4.030167,4.009983,4.023875,4.015908,4.002213,4.009255,4.003400
81043,2750,4.131424,4.084625,4.386908,4.102527,4.128737,4.148904,4.048154,4.017240,4.011070,4.050749,...,4.075637,4.011281,4.106869,4.028906,4.005785,4.019627,4.012532,4.006849,4.016539,4.007771


# Save out raw data

In [30]:
df_node.head()

,,str_0,str_1,str_2,str_3,str_4,str_5,str_6,str_7,str_8,str_9,...,sgc_190,sgc_191,sgc_192,sgc_193,sgc_194,sgc_195,sgc_196,sgc_197,sgc_198,sgc_199
bblid,scanid,,,,,,,,,,,,,,,,,,,,,
81287,2738,3647,2302,2646,1760,1617,2478,2273,2980,990,2250,...,3.68217e+10,3.04259e+10,1.07027e+11,8.096e+10,7.84375e+10,7.45103e+10,3.7598e+10,1.07025e+10,5.60843e+09,2.0806e+10
80680,2739,2626,366,1763,1052,537,1826,1711,2319,747,2288,...,3.5798e+10,3.99868e+10,1.26228e+11,2.1227e+10,3.36271e+10,1.80682e+10,1.4127e+10,2.0402e+09,3.8416e+09,5.59552e+09
81754,2740,4382,1635,1963,1737,2286,3084,2287,1280,1370,3674,...,1.70919e+12,4.15374e+12,1.14913e+12,7.89807e+11,2.77616e+12,5.69962e+11,1.43612e+11,2.86068e+10,1.67185e+10,9.6386e+09
81903,2749,3763,521,1725,1656,1204,2004,1027,1215,963,1987,...,7.48686e+10,3.09238e+10,1.98961e+11,4.93299e+10,1.13646e+11,1.47841e+11,1.23089e+11,1.57867e+10,6.12191e+09,7.23361e+09
81043,2750,2673,2388,4273,2068,2545,3955,2017,951,809,1957,...,3.34602e+11,1.71809e+11,5.96726e+11,1.2995e+11,2.81008e+11,3.93797e+11,7.94044e+10,2.26076e+10,4.38804e+09,1.02e+10


In [31]:
print(df_node.isna().any().any())
print(df_node_ac_overc.isna().any().any())
print(df_node_ac_i2.isna().any().any())

False
False
False


In [32]:
np.save(os.path.join(storedir, outfile_prefix+'A'), A)

df_node.to_csv(os.path.join(storedir, outfile_prefix+'df_node.csv'))
df_node_ac_overc.to_csv(os.path.join(storedir, outfile_prefix+'df_node_ac_overc.csv'))
df_node_ac_i2.to_csv(os.path.join(storedir, outfile_prefix+'df_node_ac_i2.csv'))
df_node_ac_overc_i2.to_csv(os.path.join(storedir, outfile_prefix+'df_node_ac_overc_i2.csv'))

df.to_csv(os.path.join(storedir, outfile_prefix+'df.csv'))

# Export for prediction

## Normalize

### Covariates

In [33]:
covs = ['ageAtScan1', 'mprage_antsCT_vol_TBV', 'dti64MeanRelRMS', 'network_density', 'streamline_count']

In [34]:
rank_r = np.zeros(len(covs),)

for i, cov in enumerate(covs):
    x = rank_int(df.loc[:,cov])
    rank_r[i] = sp.stats.spearmanr(df.loc[:,cov],x)[0]
    df.loc[:,cov] = x

print(np.sum(rank_r < 0.99))

0


### Node features

In [35]:
rank_r = np.zeros(df_node.shape[1],)

for i in tqdm(np.arange(df_node.shape[1])):
    col = df_node.iloc[:,i].name
    x = rank_int(df_node.loc[:,col])
    rank_r[i] = sp.stats.spearmanr(df_node.loc[:,col],x)[0]
    df_node.loc[:,col] = x

print(np.sum(rank_r < .99))

100%|██████████| 1000/1000 [02:06<00:00,  7.93it/s]

0


In [36]:
rank_r = np.zeros(df_node_ac_i2.shape[1],)

for i in tqdm(np.arange(df_node_ac_i2.shape[1])):
    col = df_node_ac_i2.iloc[:,i].name
    x = rank_int(df_node_ac_i2.loc[:,col])
    rank_r[i] = sp.stats.spearmanr(df_node_ac_i2.loc[:,col],x)[0]
    df_node_ac_i2.loc[:,col] = x

print(np.sum(rank_r < .99))

100%|██████████| 200/200 [00:24<00:00,  8.24it/s]

0


In [37]:
rank_r = np.zeros(df_node_ac_overc.shape[1],)

for i in tqdm(np.arange(df_node_ac_overc.shape[1])):
    col = df_node_ac_overc.iloc[:,i].name
    x = rank_int(df_node_ac_overc.loc[:,col])
    rank_r[i] = sp.stats.spearmanr(df_node_ac_overc.loc[:,col],x)[0]
    df_node_ac_overc.loc[:,col] = x

print(np.sum(rank_r < .99))

100%|██████████| 800/800 [01:34<00:00,  8.42it/s]

0


In [38]:
rank_r = np.zeros(df_node_ac_overc_i2.shape[1],)

for i in tqdm(np.arange(df_node_ac_overc_i2.shape[1])):
    col = df_node_ac_overc_i2.iloc[:,i].name
    x = rank_int(df_node_ac_overc_i2.loc[:,col])
    rank_r[i] = sp.stats.spearmanr(df_node_ac_overc_i2.loc[:,col],x)[0]
    df_node_ac_overc_i2.loc[:,col] = x

print(np.sum(rank_r < .99))

100%|██████████| 800/800 [01:34<00:00,  8.49it/s]

0


### Psychosis

In [39]:
covs = ['ageAtScan1', 'sex', 'mprage_antsCT_vol_TBV', 'dti64MeanRelRMS']
phenos = ['Overall_Psychopathology','Psychosis_Positive','Psychosis_NegativeDisorg']
print(phenos)

df_node.to_csv(os.path.join(outputdir, outfile_prefix+'X.csv'))
df_node_ac_overc.to_csv(os.path.join(outputdir, outfile_prefix+'X_ac_c.csv'))
df_node_ac_i2.to_csv(os.path.join(outputdir, outfile_prefix+'X_ac_i2.csv'))
df_node_ac_overc_i2.to_csv(os.path.join(outputdir, outfile_prefix+'X_ac_c_i2.csv'))

df.loc[:,phenos].to_csv(os.path.join(outputdir, outfile_prefix+'y.csv'))
df.loc[:,covs].to_csv(os.path.join(outputdir, outfile_prefix+'c.csv'))

covs = ['ageAtScan1', 'sex', 'mprage_antsCT_vol_TBV', 'dti64MeanRelRMS', 'streamline_count']
df.loc[:,covs].to_csv(os.path.join(outputdir, outfile_prefix+'c_sc.csv'))

['Overall_Psychopathology', 'Psychosis_Positive', 'Psychosis_NegativeDisorg']
